In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
animes = pd.read_csv('anime_filtered.csv')
users = pd.read_csv('users_filtered.csv')
animelists = pd.read_csv('animelists_filtered.csv')

In [ ]:
users.describe()

In [ ]:
# dropping users with invalid and obviously incorrect dates
users['birth_date'] = pd.to_datetime(users['birth_date'], errors='coerce')
users['last_online'] = pd.to_datetime(users['last_online'], errors='coerce')
users['join_date'] = pd.to_datetime(users['join_date'], errors='coerce')
users = users.dropna(subset=['birth_date', 'last_online', 'join_date'])
#users = users[users['last_online'].dt.year > 1950]

In [ ]:
animelists['my_last_updated'] = pd.to_datetime(animelists['my_last_updated'], unit='s')

In [ ]:
users.describe()

In [ ]:
users.head()

In [ ]:
users['gender'].hist()

In [ ]:
users['gender'].value_counts().plot.pie()

In [ ]:
users.dtypes

In [ ]:
users['birth_date'].dt.year.hist(bins=50, figsize=(16, 8))

In [ ]:
users['last_online'].dt.year.hist(bins=50, figsize=(16, 8))

In [ ]:
users['birth_date'].value_counts()

In [ ]:
users[users['birth_date'] == '1944-11-19']

In [ ]:
users['last_online'].value_counts()

In [ ]:
users[users['last_online'].dt.year == 1900].head()

In [ ]:
users['join_date'].dt.year.hist(bins=14, figsize=(16, 8))

In [ ]:
(2018 - users['birth_date'].dt.year).plot(kind='hist', bins=80, figsize=(16, 8), title='age')
plt.gca().grid('on')

In [ ]:
# some user with weird last online date
animelists[animelists['username'] == 'RedvelvetDaisuki']

In [ ]:
animelists.dtypes

In [ ]:
grouped_lists = animelists.groupby('username')

In [ ]:
%%timeit
# some users have weird last online date, but have correct last update in their animelist, so I can infer it from that.
#Now I am just trying it
#animelists[animelists['username'] == 'RedvelvetDaisuki']['my_last_updated'].max()
# 3.2 s ± 38.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
grouped_lists.get_group('RedvelvetDaisuki')['my_last_updated'].max()
# 991 µs ± 72.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
# so once grouping and then queying group per username is much faster than performing search each time

In [ ]:
#now really inferring the best estimation of last online
users.loc[users['last_online'].dt.year == 1900, 'last_online'] = users[users['last_online'].dt.year == 1900]['username'].apply(lambda x: grouped_lists.get_group(x)['my_last_updated'].max())


In [ ]:
users['last_online'].dt.year.hist(bins=11, figsize=(16, 8))

In [ ]:
animes.head()

In [ ]:
animes['episodes'].value_counts()

In [ ]:
animes[animes['anime_id'] == 21]

In [ ]:
animelists_with_eps = pd.merge(animelists, animes[['anime_id', 'episodes', 'title']], on='anime_id')

In [ ]:
animelists_with_eps.head()

In [ ]:
# just showing shitheads with incorrectly filled episodes
err_mask = (animelists_with_eps['my_watched_episodes'] > animelists_with_eps['episodes']) & (animelists_with_eps['episodes'] != 0)
animelists_with_eps[err_mask]

In [ ]:
# fixing incorrectly filled my_watched_episodes
animelists.loc[err_mask, 'my_watched_episodes'] = animelists_with_eps[err_mask]['episodes']

In [ ]:
animelists[err_mask]

In [ ]:
# example of watch time of some user with fixed episodes cound and thus watch time needs fix too.
# So watch time will be tested and reconstructed in this part
users[users['username'] == 'heeroYuy']

In [ ]:
animelists[animelists['username'] == 'heeroYuy']

In [ ]:
animes.columns

In [ ]:
animes[['anime_id', 'title', 'episodes', 'duration']].head()

In [ ]:
animes['duration'].value_counts()

In [ ]:
# calculating watch time in minutes and adding it to new column for anime
def str_to_minutes(string):
    m = re.search('((\d+) hr\.)?( ?(\d+) min\.)?( ?(\d+) sec\.)?( per ep\.)?', string)
    hours = 0 if m.group(2) is None else int(m.group(2))
    minutes = 0 if m.group(4) is None else int(m.group(4))
    seconds = 0 if m.group(6) is None else int(m.group(6))
    return hours * 60 + minutes + seconds / 60

animes['duration_min'] = animes['duration'].apply(str_to_minutes)

In [ ]:
animes[['anime_id', 'title', 'episodes', 'duration', 'duration_min']].head()

In [ ]:
lists_with_mins = pd.merge(animelists, animes[['anime_id', 'duration_min', 'title']], on='anime_id')

In [ ]:
one_user_data = lists_with_mins[lists_with_mins['username'] == 'heeroYuy']
one_user_data

In [ ]:
(one_user_data['my_watched_episodes'] * one_user_data['duration_min']).sum() / (60 * 24) # because it is in days

In [ ]:
users[users['username'] == 'heeroYuy']

In [ ]:
users[users['username'] == 'abystoma2']

In [ ]:
one_user_data = lists_with_mins[lists_with_mins['username'] == 'abystoma2']
(one_user_data['my_watched_episodes'] * one_user_data['duration_min']).sum() / (60 * 24) # because it is in days

In [ ]:
# one weird user with 2601351.02 days watches ruins the plot, ommiting him by xlim
users['user_days_spent_watching'].hist(bins=5000, figsize=(16, 8))
plt.gca().set_yscale('log')
plt.gca().set_xlim([0, 50000])


In [ ]:
users['user_days_spent_watching'].max()

In [ ]:
users[users['user_days_spent_watching'] == 2601351.02]

In [ ]:
one_user_data = lists_with_mins[lists_with_mins['username'] == 'warrose']
(one_user_data['my_watched_episodes'] * one_user_data['duration_min']).sum() / (60 * 24) # because it is in days

In [ ]:
grouped_lists = lists_with_mins.groupby('username')

def count_to_days(group):
    return (group['my_watched_episodes'] * group['duration_min']).sum() / (60 * 24)

In [ ]:
%%timeit
#(grouped_lists.get_group('warrose')['my_watched_episodes'] * grouped_lists.get_group('warrose')['duration_min']).sum() / (60 * 24)
# 2.88 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
count_to_days(grouped_lists.get_group('warrose'))
# 1.63 ms ± 58.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

In [ ]:
# now finally fixing days watch times
users['user_days_spent_watching'] = users['username'].apply(lambda x: count_to_days(grouped_lists.get_group(x)))


In [ ]:
users['user_days_spent_watching'].hist(bins=5000, figsize=(16, 8))
plt.gca().set_yscale('log')
plt.gca().set_xlim([0, 50000])


In [ ]:
animes.to_csv('anime_cleaned.csv', index=False, sep=',', encoding='utf-8')
users.to_csv('users_cleaned.csv', index=False, sep=',', encoding='utf-8')
animelists.to_csv('animelists_cleaned.csv', index=False, sep=',', encoding='utf-8')